In [0]:
import numpy
import numpy as np
import itertools as it 
import gym
import torch
import torch.nn.functional as F 
import torch.nn as nn
from torch.autograd import Variable



In [0]:
#copy

class ScaleFloatFram(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float64)

def make_env(env_name):
    env = gym.make(env_name)
    return ScaleFloatFram(env)


In [0]:
env_name = "CarRacing-v0"
env = make_env(env_name)
ob =env.reset()

In [0]:
#To understand
def one_step(i :int):
    action = env.action_space.sample()
    #action = [0,1,1]
    observation, reward, done, info = env.step(action)
    if (i%10==0):
        print(f" action : {action}")
        print(f"reward : {reward}")
        print(f"done : {done}")
        plt.show()
        plt.imshow(observation)
        print("------------------")



In [0]:
for i in range(50):
    one_step(i)

In [44]:

    # Random seed
np.random.RandomState(42)

n_episode = 4
max_horizon = 1000
#eval_steps = 10
batch_size = 10
gamma = 0.9 
#batch_size = 2
#direction frein acceleration
all_actions = np.array( [k for k in it.product([-1, 0, 1], [1, 0], [0.2, 0])])
nb_actions = len(all_actions)



In [ ]:
class Unflatten(nn.Module):
    """
    An Unflatten module receives an input of shape (N, C*H*W) and reshapes it
    to produce an output of shape (N, C, H, W).
    """
    def __init__(self, N=-1, C=3,H=96, W=96):
        super(Unflatten, self).__init__()
        self.N = N
        self.C = C
        self.H = H
        self.W = W
    def forward(self, x):
        return x.view(self.N, self.C,self.H, self.W)

class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

In [0]:
def Q_model(input_dim=96*96*3, output_dim=nb_actions, trainable=True ):
    """
    Build and return a PyTorch model implementing the architecture above.
    """
    model = nn.Sequential(
            # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
            Unflatten(batch_size, 3, 96, 96),
            #Unflatten(1, 3, 96, 96),
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=7, stride=3), #46
            nn.LeakyReLU(inplace=True, negative_slope=0.01),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2), #42
            nn.LeakyReLU(inplace=True, negative_slope=0.01),
            nn.MaxPool2d(kernel_size=2, stride=2), #21
            Flatten(),
            nn.Linear(288, 256),
            nn.LeakyReLU(inplace=True, negative_slope=0.01),
            nn.Linear(256, nb_actions),

            #nn.Linear(input_dim,48),
            #nn.LeakyReLU(0.01),
            #nn.Linear(48,48),
            #nn.LeakyReLU(0.01),
            #nn.Linear(48,output_dim)
            

            # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    )
    return model

torch.Size([2, 28224]) = 64 * 21 *21


In [67]:
#print(X.shape)



D = list()
s=ob
all_actions = np.array( [k for k in it.product([-1, 0, 1], [1, 0], [0.2, 0])])
action = all_actions[0]
reward =1
observation = ob

D.append({"st":s,"at":action,"rt":reward,"st+1":observation})
D.append({"st":s,"at":action,"rt":reward,"st+1":observation})
transitions = np.random.choice(D, size=2, replace=False)
X = np.array([ (transi["st"]) for transi in transitions])
X = torch.from_numpy(X).float()
y = np.array([ transi["rt"]for transi in transitions])

num_frame_stack = 4
pic_size=(96, 96)

dim_input_batch = (batch_size, num_frame_stack) + pic_size
dim_input_batch


state_a = np.array([ob], copy = False)
state_v = torch.tensor(state_a)
q_val = Q_model(state_v)
print(q_val)
_, act_v = torch.max(q_val, dim=1)
action = int(act_v.item())



real_data = Variable(X)
print(real_data.shape)
Q = Q_model()
answer = Q(real_data)
print(answer[0])
np.argmax(answer[0].detach().numpy())

torch.Size([2, 96, 96, 3])
tensor([-0.1008,  2.8430,  3.5826,  3.0177, -3.0547, -1.6477,  3.4126, -1.4135,
         4.4919, -0.2697, -2.5728, -2.1247], grad_fn=<SelectBackward>)


8

In [0]:
Q = Q_model()
memory = list()

def main():
    for i_episode in range(n_episode):
        observation = env.reset()
        observation =torch.from_numpy(observation).float()
        for i_step in range(max_horizon):
            print(observation.shape)
            print(Q(observation))
            #Q_current = Q(Variable(observation)))
            greedy_ind = np.argmax(Q(observation).detach().numpy())
            action = all_actions[greedy_ind]

            #action = env.action_space.sample()
            
            observation, reward, done, info = env.step(action)
            memory.append({"st":observation,"at":action,"rt":reward,"st+1":observation})
            selection = numpy.random.choice(memory, size=batch_size, replace=True)
            #print((selection))
            X = np.array([ transi["st"]for transi in selection])
            X = torch.from_numpy(X).float()
            y=torch.empty(batch_size)
            for m in range(batch_size):
                y[m] = selection[m]["rt"]
                if (done == False):
                    y[m] += gamma*Q(observation)
            Q_value = Q(X)
            print(Q_value)
            loss = F.mse_loss(Q_value, y)
            print(loss)
            
            optimizer.zero_grad()
            loss.backward()
       

    



In [0]:
main()

In [143]:
(96, 96, 3)

class Unflatten(nn.Module):
    """
    An Unflatten module receives an input of shape (N, C*H*W) and reshapes it
    to produce an output of shape (N, C, H, W).
    """
    def __init__(self, N=-1, C=3,H=96, W=96):
        super(Unflatten, self).__init__()
        self.N = N
        self.C = C
        self.H = H
        self.W = W
    def forward(self, x):
        return x.view(self.N, self.C,self.H, self.W)

In [90]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

In [40]:

        start = time()
        for batch_idx, (data, target) in enumerate(trainset_loader):
            # bring data to the computing device, e.g. GPU
            data, target = data.to(device), target.to(device)

            # forward pass
            output = model(data)
            # compute loss: negative log-likelihood
            loss = F.nll_loss(output, target)
            
            # backward pass
            # clear the gradients of all tensors being optimized.
            optimizer.zero_grad()
            # accumulate (i.e. add) the gradients from this forward pass
            loss.backward()
            # performs a single optimization step (parameter update)
            optimizer.step()
            
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, batch_idx * len(data), len(trainset_loader.dataset),
                    100. * batch_idx / len(trainset_loader), loss.item()))
            iteration += 1
            
        end = time()

NameError: name 'time' is not defined

In [47]:
%pycache
def build_dc_classifier():
    """
    Build and return a PyTorch model for the DCGAN discriminator implementing
    the architecture above.
    """
    return nn.Sequential(
        Unflatten(BATCH_SIZE, 1, 28, 28),
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1),
        nn.LeakyReLU(inplace=True, negative_slope=0.01),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1),
        nn.LeakyReLU(inplace=True, negative_slope=0.01),
        nn.MaxPool2d(kernel_size=2, stride=2),
        Flatten(),
        nn.Linear(4*4*64, 4*4*64),
        nn.LeakyReLU(inplace=True, negative_slope=0.01),
        nn.Linear(4*4*64, 1),
    )


UsageError: Line magic function `%pycache` not found.
